In [71]:
# Importing tools for data handling, visualization, and machine learning
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.impute import SimpleImputer  # Corrected import
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
import joblib
from google.colab import files
import sklearn  # To check version

In [72]:
# Checking scikit-learn version
print("scikit-learn version:", sklearn.__version__)

scikit-learn version: 1.6.1


In [73]:
# Uploading the dataset
print("Please upload Loanstatus.csv")
uploaded = files.upload()
loan_dataset = pd.read_csv('Loanstatus.csv')

Please upload Loanstatus.csv


Saving Loanstatus.csv to Loanstatus (2).csv


In [74]:
# Checking the first few rows to understand the data
print("First 5 rows of the dataset:")
print(loan_dataset.head())

First 5 rows of the dataset:
    Loan_ID Gender Married Dependents     Education Self_Employed  \
0  LP001002   Male      No          0      Graduate            No   
1  LP001003   Male     Yes          1      Graduate            No   
2  LP001005   Male     Yes          0      Graduate           Yes   
3  LP001006   Male     Yes          0  Not Graduate            No   
4  LP001008   Male      No          0      Graduate            No   

   ApplicantIncome  CoapplicantIncome  LoanAmount  Loan_Amount_Term  \
0             5849                0.0         NaN             360.0   
1             4583             1508.0       128.0             360.0   
2             3000                0.0        66.0             360.0   
3             2583             2358.0       120.0             360.0   
4             6000                0.0       141.0             360.0   

   Credit_History Property_Area Loan_Status  
0             1.0         Urban           Y  
1             1.0         Rural      

In [75]:
# Showing the number of rows and columns
print("\nDataset Shape (rows, columns):", loan_dataset.shape)



Dataset Shape (rows, columns): (614, 13)


In [76]:
# Displaying statistical summaries (like averages) for numerical columns
print("\nStatistical Summary:")
print(loan_dataset.describe())


Statistical Summary:
       ApplicantIncome  CoapplicantIncome  LoanAmount  Loan_Amount_Term  \
count       614.000000         614.000000  592.000000         600.00000   
mean       5403.459283        1621.245798  146.412162         342.00000   
std        6109.041673        2926.248369   85.587325          65.12041   
min         150.000000           0.000000    9.000000          12.00000   
25%        2877.500000           0.000000  100.000000         360.00000   
50%        3812.500000        1188.500000  128.000000         360.00000   
75%        5795.000000        2297.250000  168.000000         360.00000   
max       81000.000000       41667.000000  700.000000         480.00000   

       Credit_History  
count      564.000000  
mean         0.842199  
std          0.364878  
min          0.000000  
25%          1.000000  
50%          1.000000  
75%          1.000000  
max          1.000000  


In [77]:
# Counting missing values in each column
print("\nMissing Values:")
print(loan_dataset.isnull().sum())


Missing Values:
Loan_ID               0
Gender               13
Married               3
Dependents           15
Education             0
Self_Employed        32
ApplicantIncome       0
CoapplicantIncome     0
LoanAmount           22
Loan_Amount_Term     14
Credit_History       50
Property_Area         0
Loan_Status           0
dtype: int64


In [78]:
# Creating new features to improve model performance
# FamilyIncome: Combining ApplicantIncome and CoapplicantIncome for total income
loan_dataset['FamilyIncome'] = loan_dataset['ApplicantIncome'] + loan_dataset['CoapplicantIncome']

In [80]:
# Handling missing values
# Gender: Fill with mode
mode_gender = loan_dataset['Gender'].mode()[0]
loan_dataset['Gender'] = loan_dataset['Gender'].fillna(mode_gender)

In [81]:
# Married: Fill with mode
mode_married = loan_dataset['Married'].mode()[0]
loan_dataset['Married'] = loan_dataset['Married'].fillna(mode_married)

In [83]:
# Dependents: Fill with mode, replace '3+' with '4', convert to string
mode_dependents = loan_dataset['Dependents'].mode()[0]
loan_dataset['Dependents'] = loan_dataset['Dependents'].fillna(mode_dependents).replace('3+', '4').astype(str)

In [84]:
# Self_Employed: Fill with mode
mode_self_employed = loan_dataset['Self_Employed'].mode()[0]
loan_dataset['Self_Employed'] = loan_dataset['Self_Employed'].fillna(mode_self_employed)

In [86]:
# LoanAmount: Fill with median
median_loan_amount = loan_dataset['LoanAmount'].median()
loan_dataset['LoanAmount'] = loan_dataset['LoanAmount'].fillna(median_loan_amount)

In [87]:
# Loan_Amount_Term: Fill with median, ensure no zeros
median_loan_term = loan_dataset['Loan_Amount_Term'].median()
loan_dataset['Loan_Amount_Term'] = loan_dataset['Loan_Amount_Term'].fillna(median_loan_term)

In [88]:
# Replace any zero values with median to avoid division by zero
loan_dataset['Loan_Amount_Term'] = loan_dataset['Loan_Amount_Term'].replace(0, median_loan_term)

In [89]:
# Credit_History: Fill with mode
mode_credit_history = loan_dataset['Credit_History'].mode()[0]
loan_dataset['Credit_History'] = loan_dataset['Credit_History'].fillna(mode_credit_history)


In [90]:
# LoanAmountPerYear: Calculate after handling missing values and zeros
loan_dataset['LoanAmountPerYear'] = loan_dataset['LoanAmount'] / loan_dataset['Loan_Amount_Term']

In [91]:
# Dropping Loan_ID as it’s a unique identifier with no predictive value
loan_dataset = loan_dataset.drop(['Loan_ID'], axis=1)

In [92]:
# Checking missing values after cleaning
print("\nMissing Values After Cleaning:")
print(loan_dataset.isnull().sum())


Missing Values After Cleaning:
Gender               0
Married              0
Dependents           0
Education            0
Self_Employed        0
ApplicantIncome      0
CoapplicantIncome    0
LoanAmount           0
Loan_Amount_Term     0
Credit_History       0
Property_Area        0
Loan_Status          0
FamilyIncome         0
LoanAmountPerYear    0
dtype: int64


In [94]:
# Converting Loan_Status to numbers (Y=1, N=0)
loan_dataset['Loan_Status'] = loan_dataset['Loan_Status'].replace({'Y': 1, 'N': 0})

In [95]:
# Visualizing relationships
print("\nVisualizing Education vs Loan Status:")
sns.countplot(x='Education', hue='Loan_Status', data=loan_dataset)
plt.title('Education vs Loan Status')
plt.show()


Visualizing Education vs Loan Status:


In [96]:
print("\nVisualizing Married vs Loan Status:")
sns.countplot(x='Married', hue='Loan_Status', data=loan_dataset)
plt.title('Married vs Loan Status')
plt.show()


Visualizing Married vs Loan Status:


In [97]:
# Ensuring categorical columns are strings
categorical_columns = ['Gender', 'Married', 'Dependents', 'Education', 'Self_Employed', 'Property_Area']
for col in categorical_columns:
    loan_dataset[col] = loan_dataset[col].astype(str)

In [99]:
# Separating features (inputs) and target (output)
X = loan_dataset.drop('Loan_Status', axis=1)
y = loan_dataset['Loan_Status']

In [100]:
# Checking for NaN in X before splitting
print("\nChecking for NaN in features before splitting:")
print(X.isnull().sum())


Checking for NaN in features before splitting:
Gender               0
Married              0
Dependents           0
Education            0
Self_Employed        0
ApplicantIncome      0
CoapplicantIncome    0
LoanAmount           0
Loan_Amount_Term     0
Credit_History       0
Property_Area        0
FamilyIncome         0
LoanAmountPerYear    0
dtype: int64


In [103]:
# Identifying numerical and categorical columns
numerical_features = ['ApplicantIncome', 'CoapplicantIncome', 'LoanAmount', 'Loan_Amount_Term', 'Credit_History', 'FamilyIncome', 'LoanAmountPerYear']
categorical_features = ['Gender', 'Married', 'Dependents', 'Education', 'Self_Employed', 'Property_Area']

In [104]:
# Setting up preprocessing with SimpleImputer for numerical features
numerical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='median')),  # Impute any remaining NaN with median
    ('scaler', StandardScaler())
])
categorical_transformer = OneHotEncoder(handle_unknown='ignore', sparse_output=False)
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numerical_transformer, numerical_features),
        ('cat', categorical_transformer, categorical_features)
    ])

In [105]:
# Splitting data into 90% training and 10% testing
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, stratify=y, random_state=2)

In [106]:
# Checking for NaN in X_train
print("\nChecking for NaN in X_train:")
print(X_train.isnull().sum())


Checking for NaN in X_train:
Gender               0
Married              0
Dependents           0
Education            0
Self_Employed        0
ApplicantIncome      0
CoapplicantIncome    0
LoanAmount           0
Loan_Amount_Term     0
Credit_History       0
Property_Area        0
FamilyIncome         0
LoanAmountPerYear    0
dtype: int64


In [108]:
# Creating pipeline with preprocessing and SVM classifier
model = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('classifier', SVC(kernel='linear', random_state=2))
])

In [109]:
# Training the model
print("\nTraining the model...")
model.fit(X_train, y_train)
print("Model training complete.")


Training the model...
Model training complete.


In [110]:
# Predicting on test data
y_pred = model.predict(X_test)

In [111]:
# Evaluating the model
print("\nModel Evaluation:")
print(f"Accuracy: {accuracy_score(y_test, y_pred):.4f}")
print("\nClassification Report:")
print(classification_report(y_test, y_pred))
print("\nConfusion Matrix:")
cm = confusion_matrix(y_test, y_pred)
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=['Not Approved', 'Approved'], yticklabels=['Not Approved', 'Approved'])
plt.ylabel('Actual')
plt.xlabel('Predicted')
plt.title('Confusion Matrix')
plt.show()


Model Evaluation:
Accuracy: 0.8065

Classification Report:
              precision    recall  f1-score   support

           0       0.82      0.47      0.60        19
           1       0.80      0.95      0.87        43

    accuracy                           0.81        62
   macro avg       0.81      0.71      0.74        62
weighted avg       0.81      0.81      0.79        62


Confusion Matrix:


In [112]:
# Saving the model
model_filename = 'loan_model.pkl'
joblib.dump(model, model_filename)
print(f"\nModel saved as '{model_filename}'")


Model saved as 'loan_model.pkl'


In [113]:
# Loading the model
loaded_model = joblib.load(model_filename)
print(f"\nModel loaded from '{model_filename}'")


Model loaded from 'loan_model.pkl'


In [114]:
# Defining function to predict loan status
def predict_loan_status(new_data):
    """
    Predicts loan approval for new data.
    Input: DataFrame with same columns as training data (excluding Loan_Status).
    Output: 'Approved' or 'Not Approved'.
    """
    for col in categorical_features:
        new_data[col] = new_data[col].astype(str)
    prediction = loaded_model.predict(new_data)
    return "Approved" if prediction[0] == 1 else "Not Approved"

In [115]:
# Example new applicant data
new_applicant = pd.DataFrame([{
    'Gender': 'Male',
    'Married': 'Yes',
    'Dependents': '1',
    'Education': 'Graduate',
    'Self_Employed': 'No',
    'ApplicantIncome': 5000,
    'CoapplicantIncome': 2000,
    'LoanAmount': 150,
    'Loan_Amount_Term': 360,
    'Credit_History': 1,
    'Property_Area': 'Urban',
    'FamilyIncome': 7000,
    'LoanAmountPerYear': 150 / 360
}])

In [116]:
# Predicting for new applicant
prediction_result = predict_loan_status(new_applicant)
print(f"\nPrediction for the new applicant: {prediction_result}")


Prediction for the new applicant: Approved
